In [9]:
import os
import numpy as np
import meshio
import polyscope as ps
from scipy.spatial import ConvexHull

def load_mesh(file_path):
    """Load a mesh file using meshio and return its points and tetrahedral cells."""
    mesh = meshio.read(file_path)
    points = mesh.points
    tetrahedrons = None

    for cell_block in mesh.cells:
        if cell_block.type == "tetra":
            tetrahedrons = cell_block.data
            break

    if tetrahedrons is None:
        raise ValueError(f"No tetrahedrons found in the mesh at {file_path}")

    return points, tetrahedrons

def check_tetrahedron_intersection(tet1, tet2):
    """Check if two tetrahedrons intersect using ConvexHull."""
    combined_points = np.vstack([tet1, tet2])
    hull = ConvexHull(combined_points)
    return len(hull.vertices) != len(combined_points)

def find_interpenetrating_tetrahedrons(mesh1_points, mesh1_tetras, mesh2_points, mesh2_tetras):
    """Find all interpenetrating tetrahedrons between two meshes."""
    interpenetrating_pairs = []

    for i, tet1_indices in enumerate(mesh1_tetras):
        tet1_vertices = mesh1_points[tet1_indices]

        for j, tet2_indices in enumerate(mesh2_tetras):
            tet2_vertices = mesh2_points[tet2_indices]

            if check_tetrahedron_intersection(tet1_vertices, tet2_vertices):
                interpenetrating_pairs.append((i, j))

    return interpenetrating_pairs

def main():
    # Paths to the two mesh files
    mesh1_path = os.path.join(os.getcwd(), "../../meshes/rectangle.mesh")
    mesh2_path = os.path.join(os.getcwd(), "../../meshes/tetra.msh")

    if not os.path.exists(mesh1_path):
        raise FileNotFoundError(f"Mesh 1 file not found: {mesh1_path}")

    if not os.path.exists(mesh2_path):
        raise FileNotFoundError(f"Mesh 2 file not found: {mesh2_path}")

    # Load the meshes
    mesh1_points, mesh1_tetras = load_mesh(mesh1_path)
    mesh2_points, mesh2_tetras = load_mesh(mesh2_path)

    # Find interpenetrating tetrahedrons
    interpenetrating_pairs = find_interpenetrating_tetrahedrons(
        mesh1_points, mesh1_tetras, mesh2_points, mesh2_tetras
    )

    # Initialize Polyscope
    ps.init()

    # Register the first mesh
    ps.register_volume_mesh("Mesh 1", mesh1_points, tets=mesh1_tetras, color=(0.8, 0.3, 0.3))

    # Register the second mesh
    ps.register_volume_mesh("Mesh 2", mesh2_points, tets=mesh2_tetras, color=(0.3, 0.3, 0.8))

    # Highlight intersecting tetrahedrons
    for i, j in interpenetrating_pairs:
        # Extract vertices of the intersecting tetrahedrons
        tet1_vertices = mesh1_points[mesh1_tetras[i]]
        tet2_vertices = mesh2_points[mesh2_tetras[j]]

        # Register the intersecting tetrahedrons with a distinct color
        ps.register_surface_mesh(f"Intersecting Tetrahedron {i}-{j} (Mesh 1)", tet1_vertices, [[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]], color=(1.0, 1.0, 0.0))
        ps.register_surface_mesh(f"Intersecting Tetrahedron {i}-{j} (Mesh 2)", tet2_vertices, [[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]], color=(0.0, 1.0, 1.0))

    # Show the Polyscope window
    ps.show()

if __name__ == "__main__":
    main()
